# Tensor Ring Convolutional Neural Network

By replacing convolutional kernel with tensor ring cores, tensor ring CNN is constructed.

Here is an example to use a TR-based model with `tednet`.

In [1]:
import tednet as tdt
import tednet.tnn.tensor_ring as tr

In [2]:
# Define a TR-LeNet5
model = tr.TRLeNet5(10, [6, 6, 6, 6])

compression_ration is:  0.3968253968253968
compression_ration is:  14.17233560090703
compression_ration is:  241.54589371980677
compression_ration is:  2.867383512544803


It is easy to define a tensor ring model.
